In [4]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -U vnstock
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\admin\appdata\local\temp\pip-req-build-wmpm3v_b
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit dcf6bde8d64e1a747673d24dde30dce195bc3cf1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\admin\appdata\local\temp\pip-install-lda12sy4\elegantrl_3328390ca363425f8eebfd8048efe039
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 37aac1f592e1add9f9fd37ae8db1094656009b76
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-wmpm3v_b'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\Admin\AppData\Local\Temp\pip-install-lda12sy4\elegantrl_3328390ca363425f8eebfd8048efe039'


In [1]:
import pandas as pd
import numpy as np
import datetime

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

In [6]:
!pip install -U vnstock

In [2]:
## Có thể gọi các class chi tiết trong thư viện thay vì sử dụng cấp cao nhất là Vnstock
from vnstock import Listing, Quote, Company, Finance, Trading, Screener 



In [6]:
# Định nghĩa date ranges cho training và trading
train_date_start = '2015-01-10'
train_date_end = '2024-12-31'
trade_date_start = '2025-01-01'
trade_date_end = '2025-10-01'

print(f"Training period: {train_date_start} to {train_date_end}")
print(f"Trading period: {trade_date_start} to {trade_date_end}")

# Fetch data
quote = Quote(symbol='VNM', source='TCBS')
df = quote.history(start=train_date_start, end=trade_date_end, interval='1D')

print(f"\nFetched {len(df)} records")
df

Training period: 2015-01-10 to 2024-12-31
Trading period: 2025-01-01 to 2025-10-01

Fetched 2743 records

Fetched 2743 records


,time,open,high,low,close,volume
7,2015-01-12,30.03,30.03,29.16,29.16,148470
8,2015-01-13,29.16,29.74,29.01,29.74,84550
9,2015-01-14,30.03,30.03,29.16,29.45,87550
10,2015-01-15,29.45,29.45,29.01,29.16,86090
11,2015-01-16,29.16,29.16,29.01,29.16,33740
...,...,...,...,...,...,...
2745,2025-09-25,59.51,59.60,58.74,58.74,2757739
2746,2025-09-26,58.74,59.03,58.36,58.36,2494919
2747,2025-09-29,58.36,58.46,57.50,57.60,5139527
2748,2025-09-30,57.69,58.08,57.31,57.31,3035907


In [7]:
# ...existing code...
# debug: xem các cột hiện tại
print("columns:", df.columns.tolist())

# đảm bảo có cột 'tic' (tạo từ symbol/ticker hoặc từ biến quote nếu cần)
if 'tic' not in df.columns:
    if 'symbol' in df.columns:
        df['tic'] = df['symbol'].astype(str).str.lower()
    elif 'ticker' in df.columns:
        df['tic'] = df['ticker'].astype(str).str.lower()
    else:
        try:
            df['tic'] = quote.symbol.lower()   # nếu bạn có biến quote
        except Exception:
            df['tic'] = 'unknown'

# rename time -> date nếu cần và ép kiểu datetime
if 'time' in df.columns and 'date' not in df.columns:
    df = df.rename(columns={'time': 'date'})
df['date'] = pd.to_datetime(df['date'])

# map các tên OHLCV thay thế
cols_map = {}
for target in ['open','high','low','close','volume']:
    for alt in [target.upper(), target.capitalize(), target + '_price', target + '_PRICE']:
        if alt in df.columns and target not in df.columns:
            cols_map[alt] = target
if cols_map:
    df = df.rename(columns=cols_map)

# tạo cột day và sắp xếp; nếu 'tic' vẫn không có thì chỉ sort theo date
df['day'] = df['date'].dt.dayofweek
if 'tic' in df.columns:
    df = df.sort_values(['date','tic']).reset_index(drop=True)
else:
    df = df.sort_values('date').reset_index(drop=True)

desired = ['date','open','high','low','close','volume','tic','day']
df = df[[c for c in desired if c in df.columns]]

df.head()


columns: ['time', 'open', 'high', 'low', 'close', 'volume']


,date,open,high,low,close,volume,tic,day
0,2015-01-12,30.03,30.03,29.16,29.16,148470,vnm,0
1,2015-01-13,29.16,29.74,29.01,29.74,84550,vnm,1
2,2015-01-14,30.03,30.03,29.16,29.45,87550,vnm,2
3,2015-01-15,29.45,29.45,29.01,29.16,86090,vnm,3
4,2015-01-16,29.16,29.16,29.01,29.16,33740,vnm,4


In [10]:
# Ensure core columns exist and are in correct format
print("="*60)
print("Preparing data for technical indicators")
print("="*60)

required = ['date','tic','close','open','high','low','volume']
print("Current columns:", df.columns.tolist())

# Verify required columns
for c in ['date','tic','close','open','high','low','volume']:
    if c not in df.columns:
        raise RuntimeError(f"Missing required column: {c}")

# Convert data types
df['date'] = pd.to_datetime(df['date'])
df['tic'] = df['tic'].astype(str).str.lower()

# Ensure numeric columns
for col in ['open','high','low','close','volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove any NaN in critical columns
print(f"\nBefore cleanup - Rows: {len(df)}")
df = df.dropna(subset=['date','tic','close','open','high','low'])
print(f"After cleanup - Rows: {len(df)}")

# Check ticker group sizes
min_needed = 60
grp_sizes = df.groupby('tic').size()
print(f"\nTicker group sizes: {grp_sizes.to_dict()}")

short_tics = grp_sizes[grp_sizes < min_needed].index.tolist()
if short_tics:
    print(f"⚠ Dropping {len(short_tics)} tickers with <{min_needed} rows: {short_tics}")
    df = df[~df['tic'].isin(short_tics)].copy()
    df = df.reset_index(drop=True)
    print(f"Remaining rows: {len(df)}")

# Sort data properly (CRITICAL for FinRL)
df = df.sort_values(['tic', 'date']).reset_index(drop=True)

print(f"\nFinal shape before indicators: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Calculate technical indicators manually using stockstats
print(f"\n{'='*60}")
print("Calculating technical indicators manually...")
print(f"{'='*60}")

# Install stockstats if needed
try:
    from stockstats import StockDataFrame as Sdf
except ImportError:
    print("Installing stockstats...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'stockstats'])
    from stockstats import StockDataFrame as Sdf

# Process each ticker separately
processed_dfs = []

for tic in df['tic'].unique():
    print(f"\nProcessing {tic}...")
    tic_df = df[df['tic'] == tic].copy()
    
    # Prepare for stockstats (needs specific column names)
    stock_df = tic_df[['date', 'open', 'high', 'low', 'close', 'volume']].copy()
    stock_df = stock_df.set_index('date')
    
    # Convert to StockDataFrame
    stock = Sdf.retype(stock_df)
    
    # Calculate indicators
    try:
        # MACD
        stock['macd']  # This will auto-calculate
        tic_df['macd'] = stock['macd'].values
        
        # RSI
        stock['rsi_14']  # RSI with 14-day period
        tic_df['rsi'] = stock['rsi_14'].values
        
        # CCI
        stock['cci']
        tic_df['cci'] = stock['cci'].values
        
        # DX (Directional Index)
        stock['dx']
        tic_df['dx'] = stock['dx'].values
        
        # ATR (Average True Range)
        stock['atr']
        tic_df['atr'] = stock['atr'].values
        
        print(f"  ✓ Added: macd, rsi, cci, dx, atr")
        
    except Exception as e:
        print(f"  ✗ Error calculating indicators: {e}")
        # Fill with NaN if calculation fails
        for ind in ['macd', 'rsi', 'cci', 'dx', 'atr']:
            if ind not in tic_df.columns:
                tic_df[ind] = np.nan
    
    processed_dfs.append(tic_df)

# Combine all tickers
processed_df = pd.concat(processed_dfs, ignore_index=True)

# Sort again
processed_df = processed_df.sort_values(['date', 'tic']).reset_index(drop=True)

print(f"\n{'='*60}")
print("Technical indicators calculation complete!")
print(f"{'='*60}")
print(f"Processed shape: {processed_df.shape}")
print(f"Columns: {processed_df.columns.tolist()}")

# Check for NaN values
print(f"\nNaN values per column:")
for col in processed_df.columns:
    nan_count = processed_df[col].isna().sum()
    if nan_count > 0:
        print(f"  {col}: {nan_count} ({nan_count/len(processed_df)*100:.1f}%)")

# Drop rows with NaN in indicators (usually first few rows)
print(f"\nBefore dropping NaN: {len(processed_df)} rows")
processed_df = processed_df.dropna()
print(f"After dropping NaN: {len(processed_df)} rows")

print(f"\n{'='*60}")
print("Data preprocessing complete!")
print(f"{'='*60}")

Preparing data for technical indicators
Current columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day']

Before cleanup - Rows: 2743
After cleanup - Rows: 2743

Ticker group sizes: {'vnm': 2743}

Final shape before indicators: (2743, 8)
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day']
Date range: 2015-01-12 00:00:00 to 2025-10-01 00:00:00

Calculating technical indicators manually...

Processing vnm...
  ✓ Added: macd, rsi, cci, dx, atr

Technical indicators calculation complete!
Processed shape: (2743, 13)
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day', 'macd', 'rsi', 'cci', 'dx', 'atr']

NaN values per column:
  rsi: 1 (0.0%)
  cci: 1 (0.0%)
  dx: 1 (0.0%)

Before dropping NaN: 2743 rows
After dropping NaN: 2742 rows

Data preprocessing complete!
  ✓ Added: macd, rsi, cci, dx, atr

Technical indicators calculation complete!
Processed shape: (2743, 13)
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'tic'

In [11]:
processed_df

,date,open,high,low,close,volume,tic,day,macd,rsi,cci,dx,atr
1,2015-01-13,29.16,29.74,29.01,29.74,84550,vnm,1,0.013013,100.000000,66.666667,100.000000,0.797407
2,2015-01-14,30.03,30.03,29.16,29.45,87550,vnm,2,0.007588,65.000000,100.000000,38.095238,0.823419
3,2015-01-15,29.45,29.45,29.01,29.16,86090,vnm,3,-0.005536,47.206704,-133.333333,2.204929,0.716661
4,2015-01-16,29.16,29.16,29.01,29.16,33740,vnm,4,-0.012550,47.206704,-103.109656,2.204929,0.585941
5,2015-01-19,29.16,29.16,28.87,29.01,60950,vnm,0,-0.022945,40.548192,-99.904489,28.230517,0.527051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,2025-09-25,59.51,59.60,58.74,58.74,2757739,vnm,3,0.379627,49.292068,-24.688327,4.726049,1.269507
2739,2025-09-26,58.74,59.03,58.36,58.36,2494919,vnm,4,0.278358,47.313934,-51.399594,12.381422,1.226685
2740,2025-09-29,58.36,58.46,57.50,57.60,5139527,vnm,0,0.135217,43.549686,-93.694795,27.577411,1.207637
2741,2025-09-30,57.69,58.08,57.31,57.31,3035907,vnm,1,-0.001606,42.170980,-112.314100,30.643759,1.176377


In [12]:
# Split data into training and trading sets
print("="*60)
print("Splitting data into train and trade sets")
print("="*60)

print(f"Date ranges:")
print(f"  Training: {train_date_start} to {train_date_end}")
print(f"  Trading: {trade_date_start} to {trade_date_end}")

train = data_split(processed_df, train_date_start, train_date_end)
trade = data_split(processed_df, trade_date_start, trade_date_end)

print(f"\nSplit results:")
print(f"  Train set: {len(train)} rows")
print(f"  Trade set: {len(trade)} rows")

# Save to CSV
import os
os.makedirs('data', exist_ok=True)

train.to_csv('data/train_data.csv', index=False)
trade.to_csv('data/trade_data.csv', index=False)

print(f"\n✓ Data saved:")
print(f"  data/train_data.csv ({len(train)} rows)")
print(f"  data/trade_data.csv ({len(trade)} rows)")

print(f"\n{'='*60}")
print("Data preparation complete!")
print(f"{'='*60}")

Splitting data into train and trade sets
Date ranges:
  Training: 2015-01-10 to 2024-12-31
  Trading: 2025-01-01 to 2025-10-01

Split results:
  Train set: 2556 rows
  Trade set: 183 rows

✓ Data saved:
  data/train_data.csv (2556 rows)
  data/trade_data.csv (183 rows)

Data preparation complete!
